In [1]:
import sounddevice as sd
import numpy as np
import librosa
import joblib
from scipy.io.wavfile import write

In [2]:
def record_audio(duration=3, sample_rate=48000):
    """
    Nagrywa dźwięk z mikrofonu przez określony czas.

    Args:
    - duration (int): Czas nagrywania w sekundach.
    - sample_rate (int): Próbkowanie dźwięku.

    Returns:
    - audio_data (np.ndarray): Zarejestrowane dane audio.
    - sample_rate (int): Próbkowanie dźwięku.
    """
    print("Rozpoczynam nagrywanie...")
    audio_data = sd.rec(int(duration * sample_rate), samplerate=sample_rate, channels=1)
    sd.wait()
    print("Nagrywanie zakończone.")
    
    return audio_data, sample_rate

In [3]:
def save_audio(file_path, audio_data, sample_rate):
    """
    Zapisuje dane audio do pliku WAV.

    Args:
    - file_path (str): Ścieżka do zapisu pliku audio.
    - audio_data (np.ndarray): Dane audio do zapisania.
    - sample_rate (int): Próbkowanie dźwięku.
    """
    write(file_path, sample_rate, audio_data)

In [4]:
def predict_emotion_from_audio(file_path, model):
    """
    Przewiduje emocję na podstawie pliku audio.

    Args:
    - file_path (str): Ścieżka do pliku audio.
    - model: Wytrenowany model klasyfikacji.

    Returns:
    - emotion (str): Przewidywana emocja.
    """
    try:
        y, sr = librosa.load(file_path, sr=48000)
        
        mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
        mfccs = np.mean(mfccs.T, axis=0)
        
        mfccs = mfccs.reshape(1, -1)
        
        prediction = model.predict(mfccs)
        
        label_map = {
            1: 'Neutral',
            2: 'Calm',
            3: 'Happiness',
            4: 'Sadness',
            5: 'Anger',
            6: 'Fear',
            7: 'Disgust',
            8: 'Surprise'
        }
        
        emotion = label_map.get(prediction[0], 'Unknown')
        return emotion
    
    except Exception as e:
        print(f"Error during prediction: {e}")
        return 'Error'

In [5]:
duration = 5
audio_data, sample_rate = record_audio(duration=duration)

audio_file_path = "user_recording.wav"
save_audio(audio_file_path, audio_data, sample_rate)

model = joblib.load("D:\\noc_naukowców\\emotion_recognition_model.joblib")

emotion = predict_emotion_from_audio(audio_file_path, model)
print("Przewidywana emocja:", emotion)

Rozpoczynam nagrywanie...
Nagrywanie zakończone.
Przewidywana emocja: Anger
